In [105]:
# %matplotlib widget

from __future__ import annotations

import re
from collections import defaultdict
from dataclasses import dataclass, field
from itertools import permutations, product
from math import inf
from random import choice

import matplotlib.colors as mcolors
import matplotlib.pyplot as plt
import networkx as nx
import numpy as np
import numpy.typing as npt
from mpl_toolkits.mplot3d import axes3d
from numpy import int_, object_
from numpy.typing import NDArray
from test_utilities import run_tests_params
from util import print_hex

COLORS = list(mcolors.CSS4_COLORS.keys())

<link href="style.css" rel="stylesheet"></link>

<article class="day-desc read-aloud"><h2>--- Day 8: Two-Factor Authentication ---</h2><p>You come across a door implementing what you can only assume is an implementation of <a href="https://en.wikipedia.org/wiki/Multi-factor_authentication">two-factor authentication</a> after a long game of <a href="https://en.wikipedia.org/wiki/Requirement">requirements</a> <a href="https://en.wikipedia.org/wiki/Chinese_whispers">telephone</a>.</p>
<p>To get past the door, you first swipe a keycard (no problem; there was one on a nearby desk). Then, it displays a code on a <a href="https://www.google.com/search?q=tiny+lcd&amp;tbm=isch">little screen</a>, and you type that code on a keypad. Then, presumably, the door unlocks.</p>
<p>Unfortunately, the screen has been <span title="BUT BY WHOM?!">smashed</span>. After a few minutes, you've taken everything apart and figured out how it works. Now you just have to work out what the screen <em>would</em> have displayed.</p>
<p>The magnetic strip on the card you swiped encodes a series of instructions for the screen; these instructions are your puzzle input. The screen is <em><code>50</code> pixels wide and <code>6</code> pixels tall</em>, all of which start <em>off</em>, and is capable of three somewhat peculiar operations:</p>
<ul>
<li><code>rect AxB</code> turns <em>on</em> all of the pixels in a rectangle at the top-left of the screen which is <code>A</code> wide and <code>B</code> tall.</li>
<li><code>rotate row y=A by B</code> shifts all of the pixels in row <code>A</code> (0 is the top row) <em>right</em> by <code>B</code> pixels. Pixels that would fall off the right end appear at the left end of the row.</li>
<li><code>rotate column x=A by B</code> shifts all of the pixels in column <code>A</code> (0 is the left column) <em>down</em> by <code>B</code> pixels. Pixels that would fall off the bottom appear at the top of the column.</li>
</ul>
<p>For example, here is a simple sequence on a smaller screen:</p>
<ul>
<li><p><code>rect 3x2</code> creates a small rectangle in the top-left corner:</p><pre><code>###....
###....
.......</code></pre></li>
<li><p><code>rotate column x=1 by 1</code> rotates the second column down by one pixel:</p><pre><code>#.#....
###....
.#.....</code></pre></li>
<li><p><code>rotate row y=0 by 4</code> rotates the top row right by four pixels:</p><pre><code>....#.#
###....
.#.....</code></pre></li>
<li><p><code>rotate column x=1 by 1</code> again rotates the second column down by one pixel, causing the bottom pixel to wrap back to the top:</p><pre><code>.#..#.#
#.#....
.#.....</code></pre></li>
</ul>
<p>As you can see, this display technology is extremely powerful, and will soon dominate the tiny-code-displaying-screen market.  That's what the advertisement on the back of the display tries to convince you, anyway.</p>
<p>There seems to be an intermediate check of the voltage used by the display: after you swipe your card, if the screen did work, <em>how many pixels should be lit?</em></p>
</article>


In [106]:
from tabulate import tabulate


class LittleScreen:
    def __init__(self, width: int, height: int) -> None:
        self.rows, self.cols = height, width
        self.grid = [["."] * width for _ in range(height)]

    def rect(self, width: int, height: int) -> None:
        for row, col in product(range(height), range(width)):
            self.grid[row][col] = "#"

    def rotate_row(self, row: int, shifts: int) -> None:
        row1 = self.grid[row][:]
        for col in range(self.cols):
            self.grid[row][(col + shifts) % self.cols] = row1[col]

    def rotate_col(self, col: int, shifts: int) -> None:
        column = [self.grid[row][col] for row in range(self.rows)]
        for row in range(self.rows):
            self.grid[(row + shifts) % self.rows][col] = column[row]

    def proccess(self, instructions: str) -> None:
        for inst in instructions.strip().splitlines():
            if inst.startswith("rect"):
                self.rect(*[int(i) for i in inst[5:].split("x")])
            elif inst.startswith("rotate row"):
                self.rotate_row(*[int(i) for i in inst[13:].split(" by ")])
            elif inst.startswith("rotate column"):
                self.rotate_col(*[int(i) for i in inst[16:].split(" by ")])
            else:
                raise ValueError(f"Unknown Value {inst}")

    def pixels_lit(self) -> int:
        return sum(sum(1 for p in r if p == "#") for r in self.grid)

    def __repr__(self) -> str:
        return tabulate(self.grid, tablefmt="rounded_grid")

In [107]:
ls = LittleScreen(7, 3)
ls.rect(3, 2)
ls.rotate_col(1, 1)
ls.rotate_row(0, 4)
ls.rotate_col(1, 1)

expected = """
╭───┬───┬───┬───┬───┬───┬───╮
│ . │ # │ . │ . │ # │ . │ # │
├───┼───┼───┼───┼───┼───┼───┤
│ # │ . │ # │ . │ . │ . │ . │
├───┼───┼───┼───┼───┼───┼───┤
│ . │ # │ . │ . │ . │ . │ . │
╰───┴───┴───┴───┴───┴───┴───╯
""".strip()

actual = str(ls)

assert actual == expected

In [108]:
instructions = """
rect 3x2 
rotate column x=1 by 1
rotate row y=0 by 4
rotate column x=1 by 1
"""


screen = LittleScreen(7, 3)

screen.proccess(instructions)
actual = str(screen)
assert actual == expected
assert screen.pixels_lit() == 6

In [109]:
partI_screen = LittleScreen(50, 6)
with open("../input/day8.txt") as f:
    partI_screen.proccess(f.read())

print("Part I:", partI_screen.pixels_lit())

Part I: 128


<link href="style.css" rel="stylesheet"></link>

<main>

<p>Your puzzle answer was <code>128</code>.</p><p class="day-success">The first half of this puzzle is complete! It provides one gold star: *</p>
<article class="day-desc"><h2 id="part2">--- Part Two ---</h2><p>You notice that the screen is only capable of displaying capital letters; in the font it uses, each letter is <code>5</code> pixels wide and <code>6</code> tall.</p>
<p>After you swipe your card, <em>what code is the screen trying to display?</em></p>
</article>

</main>


In [110]:
partI_screen

╭───┬───┬───┬───┬───┬───┬───┬───┬───┬───┬───┬───┬───┬───┬───┬───┬───┬───┬───┬───┬───┬───┬───┬───┬───┬───┬───┬───┬───┬───┬───┬───┬───┬───┬───┬───┬───┬───┬───┬───┬───┬───┬───┬───┬───┬───┬───┬───┬───┬───╮
│ # │ # │ # │ # │ . │ . │ # │ # │ . │ . │ . │ # │ # │ . │ . │ # │ # │ # │ . │ . │ . │ # │ # │ . │ . │ # │ # │ # │ . │ . │ # │ . │ . │ # │ . │ # │ . │ . │ . │ # │ . │ # │ # │ . │ . │ . │ # │ # │ . │ . │
├───┼───┼───┼───┼───┼───┼───┼───┼───┼───┼───┼───┼───┼───┼───┼───┼───┼───┼───┼───┼───┼───┼───┼───┼───┼───┼───┼───┼───┼───┼───┼───┼───┼───┼───┼───┼───┼───┼───┼───┼───┼───┼───┼───┼───┼───┼───┼───┼───┼───┤
│ # │ . │ . │ . │ . │ # │ . │ . │ # │ . │ # │ . │ . │ # │ . │ # │ . │ . │ # │ . │ # │ . │ . │ # │ . │ # │ . │ . │ # │ . │ # │ . │ . │ # │ . │ # │ . │ . │ . │ # │ # │ . │ . │ # │ . │ # │ . │ . │ # │ . │
├───┼───┼───┼───┼───┼───┼───┼───┼───┼───┼───┼───┼───┼───┼───┼───┼───┼───┼───┼───┼───┼───┼───┼───┼───┼───┼───┼───┼───┼───┼───┼───┼───┼───┼───┼───┼───┼───┼───┼───┼───┼───┼───┼───┼───┼───┼───┼───

In [111]:
from colorama import Fore, Back, Style

print()
for row in partI_screen.grid:
    for p in row:
        if p == "#":
            print(Back.GREEN + " ", end="")
        else:
            print(Back.BLACK + " ", end="")
    print()
print(Style.RESET_ALL)

In [112]:
# EOARGPHYAO

<link href="style.css" rel="stylesheet"></link>
<main>

<p>Your puzzle answer was <code>EOARGPHYAO</code>.</p><p class="day-success">Both parts of this puzzle are complete! They provide two gold stars: **</p>
<p>At this point, you should <a href="/2016">return to your Advent calendar</a> and try another puzzle.</p>
<p>If you still want to see it, you can <a href="8/input" target="_blank">get your puzzle input</a>.</p>
<p>You can also <span class="share">[Share<span class="share-content">on
  <a href="https://twitter.com/intent/tweet?text=I%27ve+completed+%22Two%2DFactor+Authentication%22+%2D+Day+8+%2D+Advent+of+Code+2016&amp;url=https%3A%2F%2Fadventofcode%2Ecom%2F2016%2Fday%2F8&amp;related=ericwastl&amp;hashtags=AdventOfCode" target="_blank">Twitter</a>
  <a href="javascript:void(0);" onclick="var ms; try{ms=localStorage.getItem('mastodon.server')}finally{} if(typeof ms!=='string')ms=''; ms=prompt('Mastodon Server?',ms); if(typeof ms==='string' &amp;&amp; ms.length){this.href='https://'+ms+'/share?text=I%27ve+completed+%22Two%2DFactor+Authentication%22+%2D+Day+8+%2D+Advent+of+Code+2016+%23AdventOfCode+https%3A%2F%2Fadventofcode%2Ecom%2F2016%2Fday%2F8';try{localStorage.setItem('mastodon.server',ms);}finally{}}else{return false;}" target="_blank">Mastodon</a></span>]</span> this puzzle.</p>
</main>
